# Tests for Dense Layer

To make sure that the output from the GPU is correct, you should test them against what you get with the original CPU implementation. The following code is what you get from the original implementation on the CPU.

You should do the same for any other layer you implement.

In [1]:
import numpy as np

import pyopencl as cl
import pyopencl.array as cl_array
import PyNetwork
from PyNetwork.layers.Dense_GPU import Dense_GPU

In [2]:
input_shape = (10,)
output_nodes = 100

layer = PyNetwork.layers.Dense(output_nodes, 'relu')
layer.build(input_shape)

In [3]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

layer_gpu = Dense_GPU(context = context, queue = queue, hidden_nodes = output_nodes, activation_function='relu')
layer_gpu.build(input_shape)

# Test Predict

In [4]:
n_datapoints = 20
z_in = np.random.rand(n_datapoints, *input_shape).astype(np.float32) - 0.5

a_true, z_out_true = layer.predict(z_in, output_only=False)

In [6]:
z_in_gpu = cl_array.to_device(queue, z_in)
a_true_gpu, z_out_true_gpu = layer_gpu.predict(z_in_gpu, output_only=False)

AssertionError: Cannot do matrix multiplication.

In [10]:
np.testing.assert_almost_equal(a_true_gpu, a_true, decimal=2)

ValueError: 
error during assertion:

TypeError: float() argument must be a string or a real number, not 'Array'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/numpy/testing/_private/utils.py", line 792, in assert_array_compare
    val = comparison(x, y)
  File "/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/numpy/testing/_private/utils.py", line 1042, in compare
    z = z.astype(float_)  # handle object arrays
ValueError: setting an array element with a sequence.


Arrays are not almost equal to 2 decimals
 x: array([[cl.Array(0.11032537, dtype=float32),
        cl.Array(-0.02626567, dtype=float32),
        cl.Array(0.03026463, dtype=float32), ...,...
 y: array([[ 0.11, -0.03,  0.03, ...,  0.1 ,  0.12, -0.  ],
       [ 0.18,  0.16, -0.09, ...,  0.08,  0.1 ,  0.16],
       [-0.08,  0.06,  0.09, ..., -0.04, -0.1 , -0.07],...

# Test Backprop

In [4]:
g_prime = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
new_delta = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

delta_true = layer.get_delta_backprop_(g_prime, new_delta)

# Test Weight Gradients

In [5]:
delta = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
prev_z = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

In [6]:
delta_grad_true, w_grad_true = layer.get_weight_grad_(delta, prev_z)